# 모델 저장 및 배포
### 검증을 마친 모델을 Object Storage에 저장하고 KServe를 이용하여 모델을 배포하는 실습입니다.

# 1) 패키지 설치
- 모델 저장 및 배포에 필요한 패키지를 설치합니다.

In [ ]:
%pip -q install boto3 botocore

# 2) Object Storage에 모델 저장
- Kakaocloud Object Storage에 모델을 저장합니다.
- `S3_ACCESS_KEY`, `S3_SECRET_KEY` 에 실제 환경 변수 값으로 수정

In [ ]:
import os, boto3, botocore

ENDPOINT = "https://objectstorage.kr-central-2.kakaocloud.com"
REGION   = "kr-central-2"
BUCKET   = "models" # 버킷 이름
PREFIX   = "gender_predict" # 버킷 안 폴더 이름
LOCAL    = "./models/gender/model.joblib" # notebook의 모델 경로

AWS_ACCESS_KEY_ID = "{S3_ACCESS_KEY}"
AWS_SECRET_ACCESS_KEY = "{S3_SECRET_KEY}"

session = boto3.session.Session()
s3 = session.client(
    "s3",
    endpoint_url=ENDPOINT,
    region_name=REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    config=botocore.config.Config(
        s3={"addressing_style": "path"},
        signature_version="s3v4",
    ),
)

# 버킷 존재 확인/생성
try:
    s3.head_bucket(Bucket=BUCKET)
    print(f"Bucket '{BUCKET}' already exists.")
except Exception:
    try:
        s3.create_bucket(Bucket=BUCKET)
    except Exception:
        s3.create_bucket(
            Bucket=BUCKET,
            CreateBucketConfiguration={"LocationConstraint": REGION}
        )
    print(f"Bucket '{BUCKET}' created.")

# 업로드
assert os.path.exists(LOCAL), f"로컬 파일 없음: {LOCAL}"
key = f"{PREFIX}/model.joblib"
s3.upload_file(LOCAL, BUCKET, key)
print(f"Uploaded: s3://{BUCKET}/{key}")

# 목록 확인
resp = s3.list_objects_v2(Bucket=BUCKET, Prefix=f"{PREFIX}/")
for obj in resp.get("Contents", []):
    print(obj["Key"], obj["Size"])

# 이후 Bash 셀에서 재사용할 수 있도록 .env 파일 생성
with open("kserve_s3_creds.env", "w", encoding="utf-8") as f:
    f.write(f'KEY_ID="{AWS_ACCESS_KEY_ID}"\n')
    f.write(f'SECRET="{AWS_SECRET_ACCESS_KEY}"\n')
print("Saved creds -> kserve_s3_creds.env")


# 3) KServe 리소스 생성
- KServe가 Object Storage에 있는 모델을 읽기 위한 리소스를 생성합니다.

In [ ]:
%%bash
set -euo pipefail

NAMESPACE=kbm-u-kubeflow-tutorial

source kserve_s3_creds.env
: "${KEY_ID:?KEY_ID missing}"
: "${SECRET:?SECRET missing}"

TS=$(date +%s)
SECRET_NAME="kakaos3-credentials-${TS}"
SA_NAME="sa-kakaos3-${TS}"

# 1) Secret 생성 (create만 사용)
cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: v1
kind: Secret
metadata:
  name: ${SECRET_NAME}
  annotations:
    serving.kserve.io/s3-endpoint: objectstorage.kr-central-2.kakaocloud.com
    serving.kserve.io/s3-region: kr-central-2
    serving.kserve.io/s3-usehttps: "1"
    serving.kserve.io/s3-verifyssl: "1"
    serving.kserve.io/s3-usevirtualbucket: "0"   # path-style 권장
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: "${KEY_ID}"
  AWS_SECRET_ACCESS_KEY: "${SECRET}"
EOF

# 2) ServiceAccount 생성 (Secret 참조)
cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: ${SA_NAME}
secrets:
  - name: ${SECRET_NAME}
EOF

# 다음 셀에서 쓰도록 파일로 저장
echo "SA_NAME=${SA_NAME}" > kserve_sa.env
echo "SECRET_NAME=${SECRET_NAME}" >> kserve_sa.env

echo "Created: SA=${SA_NAME}, SECRET=${SECRET_NAME} in ns=${NAMESPACE}"


# 4) KServe InferenceService 생성 및 모델 배포
- KServe InferenceService를 이용하여 모델을 배포합니다.

In [ ]:
%%bash
set -euo pipefail

NAMESPACE=kbm-u-kubeflow-tutorial
BUCKET='models'
PREFIX='gender_predict'

source kserve_sa.env
: "${SA_NAME:?SA_NAME missing}"

cat <<EOF | kubectl create -n "$NAMESPACE" -f -
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: gender-predict
spec:
  predictor:
    sklearn:
      protocolVersion: v2
      storageUri: s3://${BUCKET}/${PREFIX}
      resources:
        requests:
          cpu: "500m"
          memory: "1Gi"
        limits:
          cpu: "1"
          memory: "2Gi"
    serviceAccountName: ${SA_NAME}
EOF